<a href="https://colab.research.google.com/github/KomalMano/Story/blob/master/16_Hyper_parameter_Tuning_(GridSearchCV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [29]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:52]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor


In [30]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(iris.data,iris.target, test_size=0.3)

In [31]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9777777777777777

In [32]:
#1st we show the hyper parameter tunning and then how to choose the model

#When we re-run the above code the value will be change.So I can not relay on this method bcz score is change on based on my samples. For that reason use KFold Cross validation.

#Cross Fold iteration which tells u each iteration 
from sklearn.model_selection import cross_val_score

In [33]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target,cv=5) 
#CV means Fold

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [34]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [35]:
cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data, iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [38]:
#So for the other approch . U can just run a for loop
#Now I m doing the exact the same thing
import numpy as np
kernels = ['rbf','linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
  for cval in C:
    cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data,iris.target,cv=5)
    avg_scores[kval+'_'+str(cval)] = np.average(cv_scores)

avg_scores
#I get these scores.Just by looking at these values ,I just can say that .
#So these scores i can find out the optimal score using the hyper parameter tunning.
#But u can see that these approch has some issues which ia if i have 4 parameters for eg then it will have 4 loops then it will be have too many iterations . It's just not a convennient .
#Luckily sklearn provides an API called GridSearchCV which will do the exact the same thing as shown in this box code.


{'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666,
 'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668}

In [40]:
#I m going to do the same thing but u will note that we will be able to do that in single line of code.
from sklearn.model_selection import GridSearchCV

#now we will define our classifier
clf = GridSearchCV(svm.SVC(gamma='auto'), {  #my 1st thing to model
     'C': [1,10,20], # now my 2nd parameter is very important.2nd parameter is parameter Grid.
      'kernel': ['rbf','linear'] # 2nd parameter is ur kernel 
}, cv=5, return_train_score=False ) #there r other parameter in grid cv.cv means cross validation

clf.fit(iris.data,iris.target)
clf.cv_results_ 
#when u execute this u get these results.U will notice that this mean_test_score.
#cv_results r not easy to view .But luckily sklearn provides a way to download these results into a dataFrame. 


{'mean_fit_time': array([0.00068154, 0.00043335, 0.00054927, 0.00044332, 0.0006361 ,
        0.00045533]),
 'mean_score_time': array([0.00034385, 0.00027671, 0.00029593, 0.0002708 , 0.00030532,
        0.00029125]),
 'mean_test_score': array([0.98      , 0.98      , 0.98      , 0.97333333, 0.96666667,
        0.96666667]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],
 'rank_test_score': array([1, 1, 1, 4, 5, 6], dtype=int32),
 'split0_test_score': array([0.96666667, 0.96

In [44]:
df = pd.DataFrame(clf.cv_results_)
df
#here r the scores of each individual split.So we ran 5 fold cross validation 
#u have mean test score as well.
#these columns in this grid might not be useful.

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000682,0.000114,0.000344,0.000048,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000433,0.000020,0.000277,0.000018,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000549,0.000027,0.000296,0.000011,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000443,0.000035,0.000271,0.000006,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000636,0.000081,0.000305,0.000009,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000455,0.000006,0.000291,0.000020,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [47]:
#So I'm going to trim it down.
df[['param_C','param_kernel','mean_test_score']]
#So u can see that these r the possible values of param_C ...
#Based on this,I can supply 1st 3 values into my parameters to get the best performance.
#So we already did this Hyper tunning parameter.U see this how this works.


,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [48]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'refit_time_',
 'return_train_score',
 'score

In [49]:
#Now let's try for best_score 1st
clf.best_score_

0.9800000000000001

In [51]:
#It will tell me best parameters
#There r multiple params .But in GridSearchCv use best params directly
clf.best_params_
#1 issue that will happen in GridSearchCV is the computation cost.
#Our dataset is very limited but just u imagine u have million of data points into ur datasets .
#and then for parameters u have so many values.Now C values I randomly took 1 to 10.But what if I just want to try range let's say number 1 to 50.Then my computation cost go very high.
#bcz this will literaly try permutation and combination for every values in each of these parameters.


{'C': 1, 'kernel': 'rbf'}

In [54]:
#To tackle this computational problem sk lib comes up with other class called randomized search cv.
#randomized search cv not try every single permutation and combination of parameter .
#But it will try random combination of these parameter values and u choose what iteration could be .
#So let's me how it works.
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'),{
        'C':[1,10,20],
        'kernel':['rbf','linear']
    },
    cv=5,
    return_train_score=False,
    n_iter=2 # now we will try 2 parameter
)
rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,20,rbf,0.966667
1,1,linear,0.980000


In [56]:
#Now look to be Hyper Parameter Tunning
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#Now I want to show u how do u choose a best model for ur given problem


In [57]:
#Now u have to define ur parameter grid and i m just defining them as simple JSON object or simple python dictionary.
#where I m saying I want to try a sphere model with 
model_params = {
    'svm': {
        'model':svm.SVC(gamma='auto'),
        'params': {
            'C':[1,10,20],
            'kernel':['rbf','linear']
        }
    },
    'random_forest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
        }
    },
    'logistic_regression':{
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    }
}

In [59]:
#I can just intialize this dic. I just can write simple for loop
scores = []

for model_name, mp in model_params.items():
  clf = GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
  clf.fit(iris.data, iris.target)
  scores.append({
      'model': model_name,
      'best_score':clf.best_score_,
      'best_params':clf.best_params_
  })
  #So all the model in previous box run in this box 1 by 1

In [60]:
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}
